<h1>Step 1: Collecting Data</h1>

---

In [12]:
from requests_html import HTML
import urllib.robotparser
import pandas as pd
import datetime
import time
from selenium.common.exceptions import WebDriverException
from selenium import webdriver

Check file robots.txt

In [3]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.worldweatheronline.com//robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

True

Dùng thư viện Selenium để tương tác với trang web và requests_html để parse html 

In [34]:
browser = webdriver.Chrome(executable_path='chromedriver.exe')
browser.get('https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

In [36]:
f = open('weatherHCM.csv', 'w', encoding='utf-8')
f.write('Date\tTime\tWeather\tTemp\tFeels\tWind\tGust\tRain\tHumidity\tCloud\tPressure\tVis\n')

date = datetime.datetime(2009, 1, 1)
end_date = datetime.datetime(2020, 12, 21)
date_col = ''

record = ['Time', 'Weather', 'Temp', 'Feels', 'Wind', 'Gust', 'Rain', 'Humidity', 'Cloud', 'Pressure', 'Vis']
n_cols = len(record)

while date < end_date:
    date_str = date.strftime('%Y-%m-%d')
    
    # Adjust date
    input_date = browser.find_element_by_id('ctl00_MainContentHolder_txtPastDate')
    browser.execute_script("arguments[0].value = arguments[1];", input_date, date_str)
    submit_date = browser.find_element_by_id('ctl00_MainContentHolder_butShowPastWeather')
    
    try:
        submit_date.click()
    except WebDriverException:
        print(f'Error: Date {date_str}')
        continue
        
    time.sleep(1) # wait for loading the page
    
    # parse html and write to file
    html = HTML(html=browser.page_source)
    yesterday = date_col 
    date_col = html.find('.card-header',first=True).find('h5', first=True).text + ' ' + str(date.year)
    if date_col == yesterday:
        continue
        
    date += datetime.timedelta(days=1)
        
    cols = html.find('.col')[20:108]
    
    i = 0
    for col in cols:
        record[i % n_cols] = col.text
        i += 1

        if i % n_cols == 0:
            f.write(f'{date_col}\t{record[0]}\t{record[1]}\t{record[2]}\t{record[3]}\t{record[4]}\t{record[5]}\t{record[6]}\t{record[7]}\t{record[8]}\t{record[9]}\t{record[10]}\n')
    
f.close()